In [1]:
import tensorflow as tf
import logging
from model import cnn_model_fn
import numpy as np
from sklearn.model_selection import train_test_split

/Users/sudatakamune/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    input_layer = tf.reshape(features['x'], [-1, 32, 32, 1])

    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu,
        name='conv1')

    conv2 = tf.layers.conv2d(
        inputs=conv1,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu,
        name='conv2')

    pool1 = tf.layers.max_pooling2d(
        inputs=conv2,
        pool_size=[2, 2],
        strides=2,
        name='pool1')

    conv3 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu,
        name='conv3')

    conv4 = tf.layers.conv2d(
        inputs=conv3,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu,
        name='conv4')

    for i in range(1, 5):
        tf.summary.histogram(
            'layer{}/bias'.format(i),
            tf.get_collection(
                tf.GraphKeys.GLOBAL_VARIABLES,
                'conv{}/bias'.format(i))[0])
        tf.summary.histogram(
            'layer{}/weights'.format(i),
            tf.get_collection(
                tf.GraphKeys.GLOBAL_VARIABLES,
                'conv{}/kernel'.format(i))[0])

    pool2 = tf.layers.max_pooling2d(
        inputs=conv4,
        pool_size=[2, 2],
        strides=2,
        name='pool2')

    pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])

    dense = tf.layers.dense(
                inputs=pool2_flat,
                units=1024,
                activation=tf.nn.relu,
                name='dense')

    dropout = tf.layers.dropout(
        inputs=dense,
        rate=0.5,
        training=mode == tf.estimator.ModeKeys.TRAIN,
        name='dropout')

    logits = tf.layers.dense(
        inputs=dropout,
        units=75,
        name='logits')

    predictions = {
        'classes': tf.argmax(input=logits, axis=1),
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(
                   mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(
            labels=labels, predictions=predictions['classes'])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [8]:
model_dir = './model/etf8g_convnet_model'

In [9]:
etf8g_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir=model_dir)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model/etf8g_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10fbc7390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
hira = np.load('hira.npz')
train_data, eval_data, train_labels, eval_labels = train_test_split(
    hira['data'], hira['labels'], stratify=hira['labels'], random_state=0)

In [10]:
tf.logging.set_verbosity(tf.logging.INFO)
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log,
    every_n_iter=50)
summary_hook = tf.train.SummarySaverHook(
    save_steps=500,
    output_dir=model_dir,
    scaffold=tf.train.Scaffold(summary_op=tf.summary.merge_all()))

In [11]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

In [15]:
for i in range(60):
    etf8g_classifier.train(
        input_fn=train_input_fn,
        steps=500,
        hooks=[logging_hook, summary_hook])

    eval_results = etf8g_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/etf8g_convnet_model/model.ckpt-30004
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 30005 into ./model/etf8g_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.         0.         0.         ... 0.         0.         0.00000004]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
INFO:tensorflow:loss = 0.0039216373, step = 30005
INFO:tensorflow:probabilities = [[0.         0.         0.  

KeyboardInterrupt: 